In [ ]:
# https://www.youtube.com/watch?v=c8K4QHzaOrU&ab_channel=NeuralHackswithVasanth

In [ ]:
#!pip install langchain-groq

In [ ]:
#!pip install protobuf==3.20.0

In [ ]:
# !pip uninstall google-cloud google-api-python-client
# !pip install google-cloud google-api-python-client


In [17]:
import sqlite3
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities.sql_database import SQLDatabase
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, create_sql_query_chain

from langchain.chains.sql_database.prompt import SQL_PROMPTS
from dotenv import find_dotenv, load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [18]:

list(SQL_PROMPTS)

['crate',
 'duckdb',
 'googlesql',
 'mssql',
 'mysql',
 'mariadb',
 'oracle',
 'postgresql',
 'sqlite',
 'clickhouse',
 'prestodb']

In [19]:
SQL_PROMPTS['sqlite']

PromptTemplate(input_variables=['input', 'table_info', 'top_k'], template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the current date, if the question involves "today".\n\nUse th

In [20]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field

llm= ChatGroq(model="mixtral-8x7b-32768", api_key="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")


# SQL DB Connection

In [21]:
# Path to the SQLite database file
db_path = r"C:\Users\Sridhar\Documents\Python Scripts\3.SQL_Agent\data\sqlite-sakila.db"

# Construct the URI by manually replacing backslashes with forward slashes
db_uri = "sqlite:///" + db_path.replace("\\", "/")
db = SQLDatabase.from_uri(db_uri)

In [24]:
db_tables = db.get_usable_table_names() # List the table names in that DB
#db_tables

['actor',
 'address',
 'category',
 'city',
 'city_stats',
 'country',
 'customer',
 'film',
 'film_actor',
 'film_category',
 'film_text',
 'inventory',
 'language',
 'payment',
 'rental',
 'staff',
 'store']

In [49]:
import pandas as pd

result = {}
# Assuming db_tables is a list of table names and conn is your database connection
for table_name in db_tables:
    output1 = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
    result[table_name] = {'table_name': table_name, 'columns': output1.columns.tolist()}

# Convert the dictionary to a list of dictionaries
data = [{'table_name': table_name, 'columns': result[table_name]['columns']} for table_name in result]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

       table_name                                            columns
0           actor     [actor_id, first_name, last_name, last_update]
1         address  [address_id, address, address2, district, city...
2        category                   [category_id, name, last_update]
3            city           [city_id, city, country_id, last_update]
4      city_stats                   [city_name, population, country]
5         country                 [country_id, country, last_update]
6        customer  [customer_id, store_id, first_name, last_name,...
7            film  [film_id, title, description, release_year, la...
8      film_actor                   [actor_id, film_id, last_update]
9   film_category                [film_id, category_id, last_update]
10      film_text                      [film_id, title, description]
11      inventory     [inventory_id, film_id, store_id, last_update]
12       language                   [language_id, name, last_update]
13        payment  [payment_id, cu

,table_name,columns
0,actor,"[actor_id, first_name, last_name, last_update]"
1,address,"[address_id, address, address2, district, city..."
2,category,"[category_id, name, last_update]"
3,city,"[city_id, city, country_id, last_update]"
4,city_stats,"[city_name, population, country]"
5,country,"[country_id, country, last_update]"
6,customer,"[customer_id, store_id, first_name, last_name,..."
7,film,"[film_id, title, description, release_year, la..."
8,film_actor,"[actor_id, film_id, last_update]"
9,film_category,"[film_id, category_id, last_update]"


In [23]:
def connect_to_sqlite(db_file):
    conn = sqlite3.connect(db_file)
    return conn
    
# Example usage
conn = connect_to_sqlite(db_path)

# Displaying all the tables in the Sakila Database
tables = pd.read_sql("""

SELECT *
FROM sqlite_master
WHERE type = 'table';

""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,actor,actor,2,CREATE TABLE actor (\n actor_id numeric NOT N...
1,table,country,country,5,CREATE TABLE country (\n country_id SMALLINT ...
2,table,city,city,7,"CREATE TABLE city (\n city_id int NOT NULL,\n..."
3,table,address,address,10,CREATE TABLE address (\n address_id int NOT N...
4,table,language,language,13,CREATE TABLE language (\n language_id SMALLIN...
5,table,category,category,15,CREATE TABLE category (\n category_id SMALLIN...
6,table,customer,customer,19,CREATE TABLE customer (\n customer_id INT NOT...
7,table,film,film,24,"CREATE TABLE film (\n film_id int NOT NULL,\n..."
8,table,film_actor,film_actor,28,CREATE TABLE film_actor (\n actor_id INT NOT ...
9,table,film_category,film_category,33,CREATE TABLE film_category (\n film_id INT NO...


In [52]:
sample = pd.read_csv("database_table_descriptions.csv")

In [53]:
sample

,Table,Description
0,productlines,Stores information about the different product...
1,products,Contains details of each product sold by the c...
2,offices,"Holds data on the company's sales offices, inc..."
3,employees,"Stores information about employees, including ..."
4,customers,"Captures data on customers, including customer..."
5,payments,"Records payments made by customers, tracking t..."
6,orders,"Details each sales order placed by customers, ..."
7,orderdetails,Describes individual line items for each sales...


In [55]:
sample.Description.values

array(['Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines.',
       'Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP. Linked to the productlines table.',
       "Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.",
       'Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reportsTo attribute.',
       'Captures data on customers, including customer number, name, contact details, address, assigned sales rep, and credit limit. Central to managing customer relationships and

# Single Tabel

In [ ]:
# generate_query = create_sql_query_chain(llm, db)
# query = generate_query.invoke({"question": "what are the cities"})
# # "what is price of `1968 Ford Mustang`"
# print(query)

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
# execute_query.invoke(query)


## Many tables

In [ ]:
class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# Dynamic Relevant Table Selection

In [ ]:

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details += f"Table Name: {row['Table']}\nTable Description: {row['Description']}\n\n"

    return table_details

In [ ]:
table_details = get_table_details()
print(table_details)

In [ ]:
# Create the Improved Prompt
table_details_prompt = f"""Return the names of SQL tables that are MOST LIKELY to be relevant to the user question. \
The tables are:

{table_details}

Based on the information provided, include tables where at least 70% of the data is relevant to the user query.

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""


In [ ]:
table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)

In [ ]:
# Invoke the chain with the input query
try:
    tables = table_chain.invoke({"input": "give me the list customers"})
    print(tables)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

In [ ]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables



In [ ]:
select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables


In [ ]:
seleted_table =select_table.invoke({"question": "give me the list customers"})


In [ ]:
select_table

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run."), # Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
         # few_shot_prompt,
         # ("human", "{input}"),
     ]
 )
print(final_prompt.format(input= "give me the list products with more quantity",table_info="city"))
generate_query = create_sql_query_chain(llm, db)


In [ ]:
 answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )
rephrase_answer = answer_prompt | llm | StrOutputParser()


In [ ]:
#rephrase_answer

In [ ]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
#| rephrase_answer
)
response= chain.invoke({"question": "give me the list customer"})

In [ ]:
# chain = (
# RunnablePassthrough.assign(table_names_to_use=select_table) |
# RunnablePassthrough.assign(query=generate_query).assign(
#     result=itemgetter("query") | execute_query
# )
# | rephrase_answer
# )
# chain.invoke({"question": "give me the list products with more quantity"})

In [ ]:
# table_names= db.get_usable_table_names()
# table_names= 'city' #'city\nactor\naddress\ncategory'

In [ ]:
# table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
# The tables are:

# {table_names}

# Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

In [ ]:
# table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)


In [ ]:
# table_chain.invoke({"input": "show me the list of cities"})  # give me details of customer and their order count


In [ ]:
# table_chain.invoke({"input": "what are the cities"})

In [ ]:
# def get_tables(tables: List[Table]) -> List[str]:
#     tables  = [table.name for table in tables]
#     return tables

# select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables

In [ ]:
 # few_shot_prompt = FewShotChatMessagePromptTemplate(
 #     example_prompt=example_prompt,
 #     examples=examples,
 #     # input_variables=["input","top_k"],
 #     input_variables=["input"],
 # )

In [ ]:
print(response)

# Done

In [ ]:
 examples = [
     {
         "input": "List all customers in France with a credit limit over 20,000.",
         "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
     },
     {
         "input": "Get the highest payment amount made by any customer.",
         "query": "SELECT MAX(amount) FROM payments;"
     },
    
 ]

In [ ]:
 example_prompt = ChatPromptTemplate.from_messages(
     [
         ("human", "{input}\nSQLQuery:"),
         ("ai", "{query}"),
     ]
 )

In [ ]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
     example_prompt=example_prompt,
     examples=examples,
     # input_variables=["input","top_k"],
     input_variables=["input"],
 )
print(few_shot_prompt.format(input1="How many products are there?"))

In [ ]:
 final_prompt = ChatPromptTemplate.from_messages(
     [
         ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
         few_shot_prompt,
         ("human", "{input}"),
     ]
 )

In [ ]:
generate_query = create_sql_query_chain(llm, db,final_prompt)

In [ ]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cities are there ?"})

In [ ]:
chain.invoke({"question": "How many cutomers with order count more than 5"})

# Langchain SQL Query Generator

#### can we edit the context to scope the tables to look into?

In [ ]:
# def chain_create(db):
#     """SQL query chain to execute queries
#     """
#     llm= ChatGroq(model="mixtral-8x7b-32768", api_key="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")
    
#     # Create the SQL query chain with the specified language model and database connection
#     chain = create_sql_query_chain(llm, db)
    
#     return chain

In [ ]:
chain= chain_create(db)

In [ ]:
chain.get_prompts()[0].pretty_print()

In [ ]:
# context = db.get_context()
# # print(list(context))
# print(context["table_info"])

In [ ]:
#chain.get_prompts()[0]

In [ ]:
len(context["table_info"])

In [ ]:
# prompt_with_context = chain.get_prompts()[0].partial(table_info=context["table_info"])
# print(prompt_with_context.pretty_repr()[:1500])

# Query Validation
https://python.langchain.com/v0.1/docs/use_cases/sql/query_checking/

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)

llm= ChatGroq(model="mixtral-8x7b-32768", api_key="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")
    
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [ ]:
full_chain

In [ ]:
query = full_chain.invoke(
    {
        "question": "i want to know every payment detail of Customer ID 408"
    }
)
query

In [ ]:
db.run(query)

In [ ]:
system = """You are a {dialect} expert. Given an input question, creat a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer
prompt.pretty_print()

In [ ]:
query = chain.invoke(
    {
        "question": "i want to know every payment detail of Customer ID 408."
    }
)
query

In [ ]:
db.run(query)

In [ ]:
def sql_infer(db, chain, user_question):
    sql_query = chain.invoke({"question": user_question})
    print(sql_query)
    result = db.run(sql_query)
    print('Query: ',result)

    answer_prompt = PromptTemplate.from_template(
                        """ Given the following user question, corresponding SQL query, and the SQL result,
                        generate a proper reply to give to user
                        
                        Question: {question}
                        SQL Query: {query}
                        SQL Result: {result}
                        Answer: """
    )
    llm_model = ChatGroq(model="mixtral-8x7b-32768", api_key="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")
    llm= LLMChain(llm= llm_model, prompt = answer_prompt)
    ans = llm(inputs={"question": user_question, "query": sql_query, "result":result})
    return ans['text']
    

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
question="show me few records of 'sqlite_master'"

out= sql_infer(db, chain, question)

In [ ]:
one = db.run("SELECT * FROM sqlite_master LIMIT 5;")

In [ ]:
print(one)

In [ ]:
import pandas as pd
pd.DataFrame(one)

In [ ]:
print(out)